In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

import numpy as np
import pandas as pd


AttributeError: module 'pandas' has no attribute 'compat'

In [6]:
# データの前処理
batch_size = 1 # データの大きさ reshapeしてからshapeの形求める感じでやろう
timesteps = 256
input_dim = 3 # acc (x, y, z)
num_classes = 5 # 分類数

train_x, train_y, test_x, test_y = (1, 1, 1, 1)

In [7]:
input_layer = Input( shape=( timesteps, input_dim) )
lstm1 = LSTM(units=256, activation="relu", kernel_initializer="he_normal", return_sequences=True, dropout=0.1)( input_layer )
lstm2 = LSTM(units=256, activation="relu", kernel_initializer="he_normal", return_sequences=True, dropout=0.1)( lstm1 )
lstm3 = LSTM(units=256, activation="relu", kernel_initializer="he_normal", return_sequences=True, dropout=0.1)( lstm2 )
dense = Dense(units=num_classes, activation="softmax")( lstm3 )

model = Model(inputs = input_layer, outputs = dense)
model.summary()

NameError: name 'Input' is not defined

In [8]:
"""
model_to_dotがうまく動かないのでこのセルは実行しないこと

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog="dot", format="svg"))
"""

'\nmodel_to_dotがうまく動かないのでこのセルは実行しないこと\n\nfrom IPython.display import SVG\nfrom tensorflow.python.keras.utils.vis_utils import model_to_dot\n\nSVG(model_to_dot(model).create(prog="dot", format="svg"))\n'